<a href="https://colab.research.google.com/github/Srishtijais16/step_demo/blob/day-4/day4_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize

# Download the 'punkt_tab' resource
nltk.download('punkt_tab')

# Sample corpus to train Word2Vec
sentences = [
    "This is a positive sentence",
    "This is another positive sentence",
    "This is a negative sentence",
    "I love this product",
    "The movie was terrible"
]

# Tokenize the sentences
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_sentences, vector_size=50, window=5, min_count=1, workers=4)

# Function to get sentence embeddings by averaging word embeddings
def get_sentence_embedding(sentence, model):
    words = word_tokenize(sentence.lower())
    word_embeddings = [model.wv[word] for word in words if word in model.wv]
    if len(word_embeddings) > 0:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

# Function to calculate cosine similarity
def calculate_similarity(text1, text2):
    embedding1 = get_sentence_embedding(text1, word2vec_model)
    embedding2 = get_sentence_embedding(text2, word2vec_model)
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Example sentences
text1 = "This is a good movie"
text2 = "This is a positive sentence"
text3 = "This is a negative sentence"

# Calculate similarities
similarity_1_2 = calculate_similarity(text1, text2)
similarity_1_3 = calculate_similarity(text1, text3)

# Print results
print(f"Similarity between '{text1}' and '{text2}': {similarity_1_2:.4f}")
print(f"Similarity between '{text1}' and '{text3}': {similarity_1_3:.4f}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Similarity between 'This is a good movie' and 'This is a positive sentence': 0.7024
Similarity between 'This is a good movie' and 'This is a negative sentence': 0.7122


In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize

# Download the 'punkt_tab' resource
nltk.download('punkt_tab')

# Sample corpus to train Word2Vec
sentences = [
    "This is a positive sentence",
    "This is another positive sentence",
    "This is a negative sentence",
    "I love this product",
    "The movie was terrible"
]

# Tokenize the sentences
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_sentences, vector_size=50, window=5, min_count=1, workers=4)

# Function to get sentence embeddings by averaging word embeddings
def get_sentence_embedding(sentence, model):
    words = word_tokenize(sentence.lower())
    word_embeddings = [model.wv[word] for word in words if word in model.wv]
    if len(word_embeddings) > 0:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

# Function to calculate cosine similarity
def calculate_similarity(text1, text2):
    embedding1 = get_sentence_embedding(text1, word2vec_model)
    embedding2 = get_sentence_embedding(text2, word2vec_model)
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Example sentences
text1 = "This is a good movie"
text2 = "This is a positive sentence"
text3 = "This is a negative sentence"

# Calculate similarities
similarity_1_2 = calculate_similarity(text1, text2)
similarity_1_3 = calculate_similarity(text1, text3)

# Print results
print(f"Similarity between '{text1}' and '{text2}': {similarity_1_2:.4f}")
print(f"Similarity between '{text1}' and '{text3}': {similarity_1_3:.4f}")

Similarity between 'This is a good movie' and 'This is a positive sentence': 0.7024
Similarity between 'This is a good movie' and 'This is a negative sentence': 0.7122


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# Import the datasets and transformers packages
!pip install datasets
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np
import pandas as pd

# Load the sms_spam dataset
# The sms_spam dataset only has a 'train' split.
# We load the 'train' split and then manually split it into train and test.
ds = load_dataset("sms_spam", split="train")

# Split the dataset into train and test
train_test_split = ds.train_test_split(test_size=0.2)  # Adjust test_size as needed
ds = {
    "train": train_test_split["train"],
    "test": train_test_split["test"],
}


# Load a pre-trained BERT model and add a classification head
model_name = "bert-base-uncased"  # Specify the desired pre-trained model name
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Binary classification (spam or not spam)
    id2label={0: "HAM", 1: "SPAM"},
    label2id={"HAM": 0, "SPAM": 1},
)

# Freeze the base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Before initializing the data collator, you need to initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name) # using the same model name

# Initialize a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./data/sms_spam_classification",
    learning_rate=3e-5,  # Adjust learning rate for experimentation
    per_device_train_batch_size=8,  # Adjust batch size for experimentation
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Train for 3 epochs
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
)


# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sms'], padding="max_length", truncation=True) # changed 'text' to 'sms'

# Applying .map() to the train and test datasets within the dictionary
tokenized_ds = {
    "train": ds["train"].map(tokenize_function, batched=True),
    "test": ds["test"].map(tokenize_function, batched=True),
}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

# Add predictions to the dataset for analysis
predictions = trainer.predict(tokenized_ds["test"])

df = pd.DataFrame(ds["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)
df["predicted_label"] = df["predicted_label"].map({0: "HAM", 1: "SPAM"})
df["label"] = df["label"].map({0: "HAM", 1: "SPAM"})

# Display mismatched predictions
mismatched = df[df["label"] != df["predicted_label"]]
pd.set_option("display.max_colwidth", None)
print("Mismatched Predictions:")
print(mismatched.head(5))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

...

<ipython-input-4-f2cd8c3051f0>:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
